<a href="https://colab.research.google.com/gist/DileepAthe/68bd4dc54828f5e0be0067a2a2e40a3a/ser001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**SPEECH EMOTION RECOGNITION**

Load the required dataset

In [1]:
!pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d dileepathe/serdataset2

In [ ]:
from zipfile import ZipFile
file_name = "serdataset2.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

traininig_folder = '/content/SER Dataset/Dataset/train/';
testing_folder = '/content/SER Dataset/Dataset/test/';
validation_folder = '/content/SER Dataset/val/';

classes_id = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'];

# create a data generator

datagen = ImageDataGenerator(rescale=1/255)


train_it = datagen.flow_from_directory(traininig_folder, classes=classes_id, batch_size=16, class_mode="sparse", target_size=(224, 224))
val_it = datagen.flow_from_directory(validation_folder, classes=classes_id, batch_size=16, class_mode="sparse", target_size=(224, 224))
test_it = datagen.flow_from_directory(testing_folder, classes=classes_id, batch_size=16, class_mode="sparse", target_size=(224, 224))

                                      
                                      
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))


## 2) Display a sample of your training set







In [ ]:
import numpy as np
x,y = train_it.next()
for i in range(5):
    image = x[i]
    plt.imshow(image[:,:,:].astype(np.float))
    plt.xlabel('Class = '+str(y[i]))
    plt.show()
    

## 3) Model Description. 


Link for info: https://keras.io/applications/#vgg16

In [ ]:
# Using Mobilenetv2 or VGG16 from keras applications. 
#from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.vgg16 import VGG16

# Define model parameters. 
#model=MobileNetV2(input_shape=(128,128,3), alpha=1.0, include_top=True, weights=None,pooling=None, classes=2)# VGG16 network
model = VGG16(include_top=True, weights=None, input_tensor=None, input_shape=(224,224,3), pooling=None, classes=7)


# Expand this cell for the model summary
model.summary()



## 4) Compile the model


WHat optimizers are? https://algorithmia.com/blog/introduction-to-optimizers

In [ ]:
from tensorflow.keras import optimizers
from keras.optimizers import Adam

model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 5) Training

In [ ]:
history =  model.fit_generator(train_it, epochs=10, validation_data=test_it)


### Plot accuracy 



In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.savefig('ACCURACY_VS_EPOCHS.png')


## 6) SAVE the model

In [ ]:
model.save("my_model.h5")

## 7) Evaluate the model

In [ ]:
# Load the model. 
model = tf.keras.models.load_model('my_model.h5')
model.evaluate_generator(test_it, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)


## 8) How to make a prediction?

In [ ]:
import cv2 
from keras.preprocessing import image

import time



img = cv2.imread('')

img_resized = cv2.resize(img, (224,224))
x = image.img_to_array(img_resized)
x = np.expand_dims(img_resized, axis=0)

t1= time.time()
features = model.predict(x) 

classes_id[int(np.argmax(features))] 

print(str(1/(time.time()-t1)))

In [ ]:
features

